In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime as dt, timedelta
from keras.models import load_model

import maria_import_export as maria
import model_mngr as modmgr
import add_indicators as indic
import split_merge as sm
import joblib


SET VARIABLES

In [ ]:
PATH_MODEL = "../../Data/Models/"

symb = "CW8"
model_type= "LSTM_CLASS"
dts_name = "DCA_CLOSE_1D_21D_V1"
lab_studied = "lab_perf_21d"
update = "202310"
timeframe=1440 #1D=1440
delta_date=3 #nb days added to the last_date
seq_len=20 
file_name=symb+"_"+model_type+"_"+dts_name+"_"+update+".h5"
scaler_name=symb+"_"+model_type+"_"+dts_name+"_"+update+"_scaler.save"

Connexion and symbol info

In [ ]:
if "con" in locals():
        maria.close_connection(con)
con = maria.get_connection()

symb_info=maria.get_symbol_info(con=con, symbol=symb)
symb_yf=symb_info.CODE_YAHOO.iloc[0]#select first line

dt_last_candle=maria.get_last_candle_date(con=con, symbol=symb,timeframe=timeframe) 
dt_delta=dt_last_candle-timedelta(days=delta_date)
print(f" {symb_yf=} {dt_last_candle=} {dt_delta=}")

Get Yahoo Data and check last months

In [ ]:
ticker_yf = yf.Ticker(symb_yf)
hist_yf = ticker_yf.history(start=dt_delta,period="1d")

hist_yf['Adj Close']=hist_yf['Close']
res=maria.load_yahoo_df_into_sql(con=con, df_yahoo=hist_yf, symbol=symb, timeframe=timeframe, del_duplicate=True)

print(maria.check_candles_last_months(con=con, symbol=symb, timeframe=timeframe))

Get last year of data and add indicators

In [ ]:
df = maria.get_candles_to_df(con=con, symbol=symb,timeframe=timeframe, only_close=True,date_start=dt_delta-timedelta(days=365))
df_indic = indic.add_indicators_to_df(con=con, df_in=df, dts_name=dts_name)
df_indic.sort_index(inplace=True)

df_clean = indic.drop_indicators_not_selected(con=con, df_in=df_indic, dts_name=dts_name, symbol=symb,label=lab_studied,algo=model_type,organize=True)

df_clean = indic.drop_indicators_by_type(
    con=con, df_in=df_clean, dts_name=dts_name, symbol=symb, ind_type=2) # drop of labels

df_clean.dropna(axis=0,how='any',inplace=True)

print(f"{df.shape=} {df_indic.shape=} {df_clean.shape=}")

Normalisation and preparation

In [ ]:
scaler = joblib.load(filename=PATH_MODEL + scaler_name)
df_norm= pd.DataFrame(scaler.transform(df_clean), columns=df_clean.columns, index=df_clean.index)

print(df_norm.describe())

x_eval_lstm,df_predict = sm.prepare_sequences_with_df(df_in=df_norm,sequence_length=seq_len)
print(f"{df_norm=} {x_eval_lstm.shape=} {df_predict.shape=}")

Load model, score and print last N days

In [ ]:
saved_model = load_model(PATH_MODEL+file_name)
arr_res = saved_model.predict(x_eval_lstm)

df_predict["predict_score"] = np.concatenate([np.array([np.nan] * (df_predict.shape[0] - len(arr_res))), np.argmax(arr_res, axis=1)]) 

print(df_predict["predict_score"][-20:])